In [ ]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import time
from datetime import datetime
import os

from sqlalchemy import create_engine

In [ ]:
# importing csv files.
crimes_19 = pd.read_csv('input_data/Chicago_Crimes_2019.csv')
crimes_20 = pd.read_csv('input_data/Chicago_Crimes_2020.csv')
crimes_21 = pd.read_csv('input_data/Chicago_Crimes_2021.csv')
fbi_code = pd.read_csv('input_data/FBI_Code.csv', encoding = "ISO-8859-1")
#fbi_code = pd.read_csv('input_data/FBI_Code.csv',  encoding= 'unicode_escape')
iucr_code = pd.read_csv('input_data/IUCR_Codes.csv', encoding = "ISO-8859-1")

In [ ]:
# merging crimes data for the last 3 years into a signle dataframe
crimes_data = pd.merge((pd.merge(crimes_19,crimes_20, how="outer")),crimes_21,how="outer")

print(len(crimes_data))

In [ ]:
#Rafael

In [ ]:
#Loading the FBI cvs file
df = fbi_code
df.head()

In [ ]:
#Loading the IUCS cvs file
df2 = iucr_code
df2.head(10)

In [ ]:
#dropping unnecessary columns
df = df.drop(df.columns[[2,3]], axis=1)
df.columns=['code', 'description']
df.head()


In [ ]:
df.columns

In [ ]:
#dropping unnecessary columns
df2 = df2.drop(df2.columns[[3,4]], axis=1)
df2.head()



In [ ]:
#Checking and Removing duplicates rows
df = df.drop_duplicates()
df.head()

#writting csv file for import into pgadmin data base
#df.to_csv(r'./Output_data/FBI_code.csv', encoding='utf-8', index=False)

In [ ]:
#Checking and Removing duplicates rows
df2 = df2.drop_duplicates()
df2.head()

In [ ]:
#Rename Columns
df2 = df2.rename(columns=({'PRIMARY DESCRIPTION':'PRIMARY_DESCRIPTION','SECONDARY DESCRIPTION':'SECONDARY_DESCRIPTION'}))
df2.head()

#writting csv file for import into pgadmin data base
#df2.to_csv(r'./Output_data/ICUS_code.csv', encoding='utf-8', index=False)

In [ ]:
#Artem
# merging crimes data for the last 3 years into a signle dataframe
crimes_data = pd.merge((pd.merge(crimes_19,crimes_20, how="outer")),crimes_21,how="outer")

#dropping unnecessary columns
crimes_data = crimes_data.drop(columns=['Latitude','Location','Longitude','X Coordinate','Y Coordinate','Updated On'])
# crimes_data.shape[0]

# removing duplicate values
crimes_data = crimes_data.drop_duplicates()
# crimes_data.shape[0]

# removing missing values
crimes_data = crimes_data.dropna()
# crimes_data.shape[0]

# Creating a function which will remove extra leading and tailing whitespace from the data.
def whitespace_remover(crimes_data):
   
    # iterating over the columns
    for i in crimes_data.columns:
         
        # checking datatype of each columns
        if crimes_data[i].dtype == 'object':
             
            # applying strip function on column
            crimes_data[i] = crimes_data[i].map(str.strip)
        else:
             
            # if condn. is False then it will do nothing.
            pass
        
# applyting whitespace_remover function on dataframe        
whitespace_remover(crimes_data)

#Rename Columns
crimes_data = crimes_data.rename(columns=({'Case Number':'Case_Number',
                                           'Location Description':'Location_Description',
                                           'Community Area':'Community_Area',
                                          'FBI Code':'FBI_Code'}))


#writting csv file for import into pgadmin data base
#crimes_data.to_csv(r'./Output_data/crimes_data.csv', encoding='utf-8', index=False)


In [ ]:
# create database connection
connection_string = "postgres:postgressql@localhost:5432/new_crime_db"
engine = create_engine(f'postgresql://{connection_string}')
# Confirm tables
engine.table_names()

In [ ]:
# rename columns to lower case 
df2.rename(columns={'IUCR':'iucr','PRIMARY_DESCRIPTION':'primary_description','SECONDARY_DESCRIPTION':'secondary_description' }, inplace=True)
crimes_data.columns=['id', 'case_number', 'date', 'block', 'iucr','location_description',
                     'arrest', 'domestic','beat','district', 'district_name', 'district_population',
                     'ward', 'community_area', 'fbi_code'
                    ]


In [ ]:
#load dataframe into database

df2.to_sql(name='iucr_codes', con=engine, if_exists='append', index=False)
df.to_sql(name='fbi_description', con=engine, if_exists='append', index=False)

In [ ]:
crimes_data.to_sql(name='crime_table', con=engine, if_exists='append', index=False)